<a href="https://colab.research.google.com/github/pedro00dk/ufpe-security-analysis/blob/master/ufpe_sa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UFPE Security Analysis

Fetching github repo to get database file.

In [10]:
!rm --recursive --force -- ./ufpe-security-analysis/
!git clone -- https://github.com/pedro00dk/ufpe-security-analysis
!ls -- ./ufpe-security-analysis/

Cloning into 'ufpe-security-analysis'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.
 README.md   ufpe_sa.ipynb  'UFPE SSI.xlsx'


Install necessary tools for loading excel

In [9]:
!pip install xlrd

    100% |████████████████████████████████| 112kB 2.7MB/s 


In [0]:
import numpy as np
import pandas as pd
import re

The database consists in an excel file with sheets per year of information. All of these sheets have to be loaded.

In [44]:
xls = pd.ExcelFile('./ufpe-security-analysis/UFPE SSI.xlsx')
dfs = xls.parse(sheet_name=xls.sheet_names)

[display(y, df.sample(2)) for y, df in dfs.items()]

'2008'

,Data,Quadrante,Local,Horário,Tipo,Descrição Sumária,Partes,Medidas
202,21.12.08,NaN,Ñinf,16h00,Colisão,Ocorrência 1071-houve uma colisão entre os véi...,Ñinf,Os condutores entraram em acordo.
77,06.03.08,NaN,Ñinf,19h20,Prestação de Socorro,Ocorrência 504 - o aluno passou mal .,Elieldes,Foi conduzido até a sua residência


'2009'

,Data,Quadrante,Local,Horário,Tipo,Descrição Sumária,Partes,Medidas
42,14.02.09,Alfa,PROACAD,20h30,Prestação de socorro,Ocorrência 1164 - Prestação de socorro ao func...,Funcionário Fernando Mendes Oliveira.,O funcionário foi levado ao hospítal Getúlio V...
11,18.01.09,Alfa,Terapia ocupacional,Ñ inf,Animal solto,Ocorrência 1121 - 01 cavalo deu um coice e der...,Segurança de terapia ocupacional Paulo Galdino.,O GTO entrou em contato com o corpo de bombeir...


'2011'

...

'2014'

,Data,Quadrante,Local,horário,Tipo,Descrição Sumária,Partes,Medidas
132,2014-04-23 00:00:00,Beta,CCEN,22h30,Registro de ocorrência,Ocorrência 7402 - segundo registro várias plac...,Ñ inf,Agentes da TKS trouxeram as placas para a DGOS.
248,2014-07-08 00:00:00,ZLI,Campus UFPE,14h35,Registro de ocorrência,Ocorrência 7653 - segundo registro faltou ener...,Ñ inf,A Celpe foi informada e solucionou o problema.


'2015'

,Data,Quadrante,Local,horário,Tipo,Descrição Sumária,Partes,Medidas,Unnamed: 8,Unnamed: 9,...,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35
436,2015-11-26,Gama,CFCH,10h35,Carro c/ Vulnerabilidade,Ocorrência 0680 TURMA “C” - segundo registro o...,Proprietário do Veículo: Ivanildo Coelho de Al...,Os documentos que estavam dentro do veículo fo...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
319,2015-07-14,Ñ inf,Ñ inf,08h00,Prestação de Socorro,"Ocorrência 0395 TURMA ""B"" - segundo registro o...",Segurança do GTO: José Manoel Borges.,O segurança foi conduzido até a UPA da Caxangá.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


[None, None, None, None, None, None, None]

In [56]:
drop_rule = re.compile(
    r'''(?ix)
    descrição \s sumária |
    medidas |
    unnamed.*
    '''
)
na_rule = re.compile(
    r'(?ix)ñ.*'
)

dfs = {
    y: df.drop([c for c in df.columns if drop_rule.match(c) != None], axis='columns')
    for y, df in dfs.items()
}
dfs = {
    y: df.replace(na_rule.pattern, np.nan, regex=True)
    for y, df in dfs.items()
}


[display(y, df.sample(10)) for y, df in dfs.items()]

'2008'

,Data,Quadrante,Local,Horário,Tipo,Partes
4,08.01.08,Gama,CFCH,06h10,Tentativa suicídio,Téo Neves da Silveira
111,30.04.08,Gama,CECINE,00h00,Prestação de Socorro,João Mota Marinho
158,16.08.08,Beta,CentroConvenções,NaN,Registro de ocorrência,NaN
193,06.12.08,NaN,NaN,17h00,Registro de ocorrência,Não sabia dizer o nome.
80,11.03.08,Delta,CTG,NaN,Colisão,Jose Walmir Arruda/ Rogério Felix dos Santos
205,27.12.08,Alfa,CEU Feminina,22h35,Roubo - ZLI,Renata Moura
167,28.09.08,Alfa,NEFD,18h20,Registro de ocorrência,Luis Carlos da Silva
12,14.01.08,NaN,NaN,14h28,Carro aberto,NaN
19,19.01.08,Alfa,NEFD,18h30,Registro de ocorrência,NaN
3,04.01.08,Beta,Arqueologia,14h00,Invasão,NaN


'2009'

,Data,Quadrante,Local,Horário,Tipo,Partes
108,24.05.09,Alfa,Reitoria/Real/Sudene,14h00,Furto,GTO
147,04.11.09,NaN,Depto Zoologia,13h00,Furto,GTO
87,19.03.09,Gama,Restaurante Universitário,23h40,Tentativa de assalto,NaN
32,05.02.09,Gama,CFCH,14h35,Registro de ocorrência,Srª Ledy Selma Ferreira.
114,06.06.09,Alfa,COS,09h30,Roubo,GTO
77,14.03.09,Gama,CFCH,21h35,Registro de ocorrência,Srº Iuri Alves de Araújo.
104,07.05.09,ZLE,Medicina -externo - parada de ônibus - BR 101,21h30,Roubo,GTO.
94,24.03.09,Alfa,Reitoria,16h00,Registro de ocorrência,João Lucas Marques e Silva / Alex André dos Sa...
91,23.03.09,Beta,CTG,NaN,Registro de ocorrência,NaN
141,18.10.09,Delta,Fisiologia,15h00,Arrombamento e Furto,GTO


'2011'

...

'2014'

,Data,Quadrante,Local,horário,Tipo,Partes
322,2014-09-08 00:00:00,Gama,Laguinho,15h00,Uso de entorpecentes,NaN
102,2014-03-27 00:00:00,NaN,Em frente a escada,09h52,Carro c/ vulnerabilidade,NaN
69,2014-02-24 00:00:00,Alfa,DGOS,15h45,Registro de ocorrência,Pai Charles Bezerra dos Santos / Jovem Ítalo H...
326,2014-09-18 00:00:00,Alfa,Reitoria,15h30,Registro de ocorrência,NaN
233,2014-06-24 00:00:00,Alfa,CIS,16h15,Registro de ocorrência,Vigilante: Adriana Nascimento.
33,2014-02-03 00:00:00,Beta,CTG,NaN,Registro de ocorrência,Professor Félix Cristian Guimarães Santos.
308,2014-08-29 00:00:00,Alfa,NEFD,22h50,Prestação de socorro,NaN
71,2014-02-25 00:00:00,Gama,Pista de Cooper,21h40,Uso de entorpecentes,Elementos sem documentos.
63,2014-02-20 00:00:00,Alfa,NEFD,21h00,Roubo ZLI,Jovens Calymom Rodrigo Pontes da Silva e Ítalo...
310,2014-08-29 00:00:00,Gama,Estacionamento do CCSA,12h30,Carro c/ Vulnerabilidade,NaN


'2015'

,Data,Quadrante,Local,horário,Tipo,Partes
12,2015-01-11,ZLE,TV Universitária,22h00,Dano ao Patrimônio,Informante: Inspetor de segurança Bione.
401,2015-10-23,Beta,CTG,17h15,Roubo ZLI,Cdadão: Olavo.
340,2015-08-14,Beta,Parada de Ônibus externa do CTG,10h55,Roubo ZLE,Funcionária: Thais Silva da Costa.
267,2015-06-05,Alfa,NEFD,09h40,Registro de Ocorrência,Professor: Gustavo Cesar de Vasconcelos.
221,2015-05-10,ZLE,Em frente ao Atacadão da Iputinga,18h00,Prestação de Socorro Mecânico,Vigilante da TKS: Rivelino / Seguranças: David...
152,2015-04-01,Delta,CCS,21h00,Registro de Ocorrência,Informante: Orlando (segurança do local).
358,2015-09-13,Beta,CIN,14h30,Registro de Ocorrência,NaN
102,2015-03-11,Alfa,NAE,22h30,Prestação de Socorro,Aluna: Rafaela Goes da Silva / Acompanhantes: ...
190,2015-04-19,Delta,Entre o CCB e o CCS,NaN,Registro de Ocorrência,Aluno: João Sérgio de Oliveira (Curso de Biome...
29,2015-01-21,Alfa,NAE,13h15,Colisão,Proprietários: Laerty Bacbino da Silva (Celta)...


[None, None, None, None, None, None, None]